In [114]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
import networkx as nx
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%run Data_center_setup.ipynb

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

[[0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0.]]


$$
\begin{aligned}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{i,t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{i,t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t} - \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{i,t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{aligned}
$$

$$
\begin{equation*}
\begin{array}{lll}
    &\min \begin{bmatrix}
0\\
0\\
0\\
0\\
\end{bmatrix}^{\intercal} 
\begin{bmatrix}
u^{\text{dis}}_{t}\\
u^{\text{chr}}_{t}\\
x^{\text{chiller}}_{j,t}\\
x^{\text{tower}}_{j,t}\\
\end{bmatrix}
+ \begin{bmatrix}
0\\
0\\
0\\
0\\
0\\
0\\
0\\
0\\
\mathbf{1}_{t}\\
\end{bmatrix}^{\intercal}
\begin{bmatrix}
p^{\text{dis}}_{t}\\
p^{\text{chr}}_{t}\\
T^{\text{zone}}_{i,t}\\
T^{\text{zone}}_{i,t+1}\\
T^{\text{sup}}_{i,t}\\
v^{\text{vent}}_{t}\\
E^{\text{B,state}}_{t}\\
E^{\text{B,state}}_{t+1}\\
e^{in}_t
\end{bmatrix} 
& \\
    s.t& 
\begin{bmatrix}
(1) & 0& 0& -\left(\beta^{\text{chilller}}_{0,j} + \beta^{\text{chilller}}_{1,j} \cdot m^{\text{chiller}}_{j,t}\right) & -\left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j} \cdot m^{\text{tower}}_{j,t}\right)\\
(2) & 0 & 0 & 0 & 0\\
(3) & 0 & 0 & 0 & 0\\
(4) & 0 & 0 & 0 & 0\\
(5) &0 & 0 & 0 & 0\\
(6) &0 & 0 & 0 & 0\\
(7) &0  & \overline{p^{\text{chr}}_{t}} & 0 & 0\\
(8) &\overline{p^{\text{dis}}_{t}}  & 0 & 0 & 0\\
(9) &1& 1 & 0 & 0\\
(10) &0 & 0 & 0 & 0\\
(11) &0 & 0 & 0 & 0\\
(12) &0 & 0 & 0 & 0\\
(13) &0 & 0 & 0 & 0\\
(14) &0 & 0 & m^{\text{chiller}}_{j,t} & 0\\
(14) &0 & 0 & 0 & m^{\text{tower}}_{j,t}\\
\end{bmatrix}
\begin{bmatrix}
u^{\text{dis}}_{t}\\
u^{\text{chr}}_{t}\\
x^{\text{chiller}}_{j,t}\\
x^{\text{tower}}_{j,t}\\
\end{bmatrix}\\+
&\begin{bmatrix}
(1) &\left(\eta^{\text{dis}}\right)^{-1}&-\eta^{\text{chr}}&
-\beta^{\text{pump}}_{1}\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i} \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}& 0 &
\beta^{\text{pump}}_{1} \frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}&
-\beta^{\text{vent}}_{t}& 0 & 0 &
\mathbf{1}_{t}\\
(2) &0&0&0&0&0&0&0&0&\mathbf{1}_{t}\\
(3) &\left(\eta^{\text{dis}}\right)^{-1}&\eta^{\text{chr}}&0&0&0&0&-\mathbf{1}_{t}&\mathbf{1}_{t}&0\\
(4) & 0& 0& 0& 0& 0& 0& \mathbf{1}_{t=0} & 0& 0 \\
(5) & 0& 0& 0& 0& 0& 0& 0 & \mathbf{1}_{t} & 0 \\
(6) & 0& 0& 0& 0& 0& 0& 0 & \mathbf{1}_{t} & 0 \\
(7) & 0 & 1& 0& 0& 0& 0& 0 & 0 & 0 \\
(8) & 1& 0& 0& 0& 0& 0& 0 & 0 & 0 \\
(9) & 0& 0& 0& 0& 0& 0& 0 & 0 & 0 \\
(10) & 0& 0& -1 - \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)}\left(\frac{\mathbf{1}_{i^{\prime}} - 1}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right) + \frac{\dot{m}_{i} c_{a}}{C^{\text{heat}}_{i}}
& 1 &
-\frac{\dot{m}_{i} c_{a}}{C^{\text{heat}}_{i}}& 0& 0 & 0 & 0 \\
(11) & 0& 0& 0& 1& 0& 0& 0 & 0 & 0 \\
(12) & 0& 0& 0& 1& 0& 0& 0 & 0 & 0 \\
(13) & 0& 0& 0& 0& 0& 1& 0 & 0 & 0 \\
(14) & 0& 0& -\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i} \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 
\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 0 & 0 & 0 \\
(15) & 0& 0& -\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i} \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 
\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 0 & 0 & 0 \\
\end{bmatrix} 
\begin{bmatrix}
p^{\text{dis}}_{t}\\
p^{\text{chr}}_{t}\\
T^{\text{zone}}_{i,t}\\
T^{\text{zone}}_{i,t+1}\\
T^{\text{sup}}_{i,t}\\
v^{\text{vent}}_{t}\\
E^{\text{B,state}}_{t}\\
E^{\text{B,state}}_{t+1}\\
e^{in}_t
\end{bmatrix} \\
&\begin{bmatrix}
(1) & = & e^{\text{sup}}_{t} + \beta^{\text{vent}}_{0} -\beta^{\text{vent}}_{1}\underline{v}^{\text{vent}} + \beta^{\text{pump}}_{0} + \beta^{\text{pump}}_{1} \frac{T^{\text{out}}_{i,t}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}+ E^{\text{Se}}_{t} - E^{\text{Solar}}_{t}\\
(2) & \geq& 0\\
(3) & = & 0\\
(4) & = & E^{\text{B,state}}_{\text{initial}}\\
(5) & \leq & \overline{\xi^B}\\
(6) & \geq & \underline{\xi^B}\\
(7) & \geq & 0\\
(8) & \geq & 0\\
(9) & \leq & 1\\
(10) & = & \frac{\theta_{i,t}}{C^{\text{heat}}_{i}}\\
(11) & \leq & T^{\text{+}}_{i,t}\\
(12) & \geq & T^{\text{-}}_{i,t}\\
(13) & \geq & \underline{v}^{\text{vent}}_{t} \\
(14) & \geq & \frac{T^{\text{out}}_{i,t}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}\\
(15) & \geq & \frac{T^{\text{out}}_{i,t}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot c^{\text{water}}_p}\\
\end{bmatrix} 
\end{array}
\end{equation*}
$$

## After the refomulation

$$
\begin{equation*}
\begin{array}{lll}
    &\min \begin{bmatrix}
0\\
0\\
0\\
0\\
\end{bmatrix}^{\intercal} 
\begin{bmatrix}
u^{\text{dis}}_{t}\\
u^{\text{chr}}_{t}\\
x^{\text{chiller}}_{j,t}\\
x^{\text{tower}}_{j,t}\\
\end{bmatrix}
+ \begin{bmatrix}
0\\
0\\
0\\
0\\
0\\
0\\
0\\
0\\
\mathbf{1}_{t}\\
\end{bmatrix}^{\intercal}
\begin{bmatrix}
p^{\text{dis}}_{t}\\
p^{\text{chr}}_{t}\\
T^{\text{zone}}_{i,t}\\
T^{\text{zone}}_{i,t+1}\\
T^{\text{sup}}_{i,t}\\
v^{\text{vent}}_{t}\\
E^{\text{B,state}}_{t}\\
E^{\text{B,state}}_{t+1}\\
e^{in}_t
\end{bmatrix} 
& \\
    s.t& 
\begin{bmatrix}
(1) & 0& 0& -\left(\beta^{\text{chilller}}_{0,j} + \beta^{\text{chilller}}_{1,j} \cdot m^{\text{chiller}}_{j,t}\right) & -\left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j} \cdot m^{\text{tower}}_{j,t}\right)\\
(3) & 0 & 0 & 0 & 0\\
(4) & 0 & 0 & 0 & 0\\
(10) &0 & 0 & 0 & 0\\
(2) & 0 & 0 & 0 & 0\\
(5) &0 & 0 & 0 & 0\\
(6) &0 & 0 & 0 & 0\\
(7) &0  & \overline{p^{\text{chr}}_{t}} & 0 & 0\\
(8) &\overline{p^{\text{dis}}_{t}}  & 0 & 0 & 0\\
(9) & -1 & -1 & 0 & 0\\
(11) &0 & 0 & 0 & 0\\
(12) &0 & 0 & 0 & 0\\
(13) &0 & 0 & 0 & 0\\
(14) &0 & 0 & m^{\text{chiller}}_{j,t} & 0\\
(15) &0 & 0 & 0 & m^{\text{tower}}_{j,t}\\
\end{bmatrix}
\begin{bmatrix}
u^{\text{dis}}_{t}\\
u^{\text{chr}}_{t}\\
x^{\text{chiller}}_{j,t}\\
x^{\text{tower}}_{j,t}\\
\end{bmatrix}\\+
&\begin{bmatrix}
(1) &\left(\eta^{\text{dis}}\right)^{-1}&-\eta^{\text{chr}}&
-\beta^{\text{pump}}_{1}\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i} \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}& 0 &
\beta^{\text{pump}}_{1} \frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}&
-\beta^{\text{vent}}_{t}& 0 & 0 &
\mathbf{1}_{t}\\
(3) &\left(\eta^{\text{dis}}\right)^{-1}&\eta^{\text{chr}}&0&0&0&0&-\mathbf{1}_{t}&\mathbf{1}_{t}&0\\
(4) & 0& 0& 0& 0& 0& 0& \mathbf{1}_{t=0} & 0& 0 \\
(10) & 0& 0& -1 - \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)}\left(\frac{\mathbf{1}_{i^{\prime}} - 1}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right) + \frac{\dot{m}_{i} c_{a}}{C^{\text{heat}}_{i}}
& 1 &
-\frac{\dot{m}_{i} c_{a}}{C^{\text{heat}}_{i}}& 0& 0 & 0 & 0 \\
(2) &0&0&0&0&0&0&0&0&\mathbf{1}_{t}\\
(5) & 0& 0& 0& 0& 0& 0& 0 & -\mathbf{1}_{t} & 0 \\
(6) & 0& 0& 0& 0& 0& 0& 0 & \mathbf{1}_{t} & 0 \\
(7) & 0 & 1& 0& 0& 0& 0& 0 & 0 & 0 \\
(8) & 1& 0& 0& 0& 0& 0& 0 & 0 & 0 \\
(9) & 0& 0& 0& 0& 0& 0& 0 & 0 & 0 \\
(11) & 0& 0& 0& -1& 0& 0& 0 & 0 & 0 \\
(12) & 0& 0& 0& 1 & 0& 0& 0 & 0 & 0 \\
(13) & 0& 0& 0& 0& 0& 1& 0 & 0 & 0 \\
(14) & 0& 0& -\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i} \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 
\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 0 & 0 & 0 \\
(15) & 0& 0& -\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i} \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 
\frac{\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot c^{\text{water}}_p}& 0& 0 & 0 & 0 \\
\end{bmatrix} 
\begin{bmatrix}
p^{\text{dis}}_{t}\\
p^{\text{chr}}_{t}\\
T^{\text{zone}}_{i,t}\\
T^{\text{zone}}_{i,t+1}\\
T^{\text{sup}}_{i,t}\\
v^{\text{vent}}_{t}\\
E^{\text{B,state}}_{t}\\
E^{\text{B,state}}_{t+1}\\
e^{in}_t
\end{bmatrix} \\
&\begin{bmatrix}
(1) & = & e^{\text{sup}}_{t} + \beta^{\text{vent}}_{0} -\beta^{\text{vent}}_{1}\underline{v}^{\text{vent}} + \beta^{\text{pump}}_{0} + \beta^{\text{pump}}_{1} \frac{T^{\text{out}}_{i,t}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}+ E^{\text{Se}}_{t} - E^{\text{Solar}}_{t}\\
(3) & = & 0\\
(4) & = & E^{\text{B,state}}_{\text{initial}}\\
(10) & = & \frac{\theta_{i,t}}{C^{\text{heat}}_{i}}\\
(2) & \geq& 0\\
(5) & \geq & -\overline{\xi^B}\\
(6) & \geq & \underline{\xi^B}\\
(7) & \geq & 0\\
(8) & \geq & 0\\
(9) & \geq & -1\\
(11) & \geq & -T^{\text{+}}_{i,t}\\
(12) & \geq & T^{\text{-}}_{i,t}\\
(13) & \geq & \underline{v}^{\text{vent}}_{t} \\
(14) & \geq & \frac{T^{\text{out}}_{i,t}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}\\
(15) & \geq & \frac{T^{\text{out}}_{i,t}v^{\text{sup}} c^{\text{air}}_p}
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot c^{\text{water}}_p}\\
\end{bmatrix} 
\end{array}
\end{equation*}
$$

Decision Var
$$
\begin{equation*}
\begin{array}{lll}
    &\min c^{\intercal}\mathbf{x}+ d^{\intercal}\mathbf{y}& \\
    s.t& A^{\prime}\mathbf{x} + G^{\prime} \mathbf{y} = \mathbf{b}^{\prime}\\
    & A\mathbf{x}+G\mathbf{y} \geq \mathbf{b}  \\
\end{array}
\end{equation*}
$$


$$
\begin{align*}
\min_{\mathbf{x}} \quad &  \mathbf{c}^{\intercal}\mathbf{x} + z_{LP}(\mathbf{x})\\
\textrm{s.t.} \quad & \mathbf{x} \in \mathbf{X}, \ \mathbf{x} \in \left\{0,1\right\}^{n} \\
\end{align*}
$$

$$
\mathbf{c} = \begin{bmatrix}
0\\
0\\
0\\
0\\
\end{bmatrix}
$$

$$
h = \begin{bmatrix}
0\\
0\\
0\\
0\\
0\\
0\\
0\\
0\\
\mathbf{1}_{t}\\
\end{bmatrix}
$$

$$
\begin{align*}
z_{LP}(\mathbf{x}) = \min_{\mathbf{y}} \quad &  \mathbf{h}^{\intercal}\mathbf{y}\\
\textrm{s.t.} \quad & \mathbf{G}^{\prime}\mathbf{y} = \mathbf{b}^{\prime} - \mathbf{A}\mathbf{x}\\
& \mathbf{G}\mathbf{y} \geq \mathbf{b} - \mathbf{A}\mathbf{x}\\
&\mathbf{y}\in \mathbb{R}^{p}_{+}
\end{align*}
$$

$$
\begin{align*}
z_{LP}(\mathbf{x}) = \max_{\mathbf{u},\mathbf{u^{\prime}}} \quad &  
{\begin{bmatrix}
\left(\mathbf{b}^{\prime} - \mathbf{A}^{\prime}\mathbf{x}\right)\\
\left(\mathbf{b} - \mathbf{A}\mathbf{x}\right) \\
\end{bmatrix}}^{\intercal}
\begin{bmatrix}
\mathbf{u}^{\prime}\\
\mathbf{u}\\
\end{bmatrix}
 \\
\textrm{s.t.} \quad & 
\begin{bmatrix}
\mathbf{G}^{\prime}\\
\mathbf{G}
\end{bmatrix}^{\intercal}
\begin{bmatrix}
\mathbf{u}^{\prime}\\
\mathbf{u}
\end{bmatrix}
\leq h\\
&\mathbf{u}\in \mathbb{R}^{p}_{+}\\
&\mathbf{u^{\prime}} \ \text{unrestricted}
\end{align*}
$$

In [115]:
## Quantum Part CQM
from dwave.system import DWaveSampler, LeapHybridSampler, LeapHybridCQMSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dimod

from dimod.binary.binary_quadratic_model import BinaryQuadraticModel, Binary, Spin, as_bqm
from dimod.discrete.discrete_quadratic_model import DiscreteQuadraticModel
#function set
%run Func_Set.ipynb



t_bits = 40
t_decimal = 4


cqm = dimod.ConstrainedQuadraticModel()
u_name_iterable = ['dischar','char']

u = [[dimod.Binary(f'u_{u_name}_{t}') for t in range(T)] for u_name in u_name_iterable]
u_coefficient = np.zeros((len(u_name_iterable), T))

x_chiller = [[dimod.Binary(f'x_chiller_#{j}_{t}') for t in range(T)] for j in range(J_chiller)]
x_chiller_coefficient = np.zeros((J_chiller, T))

x_tower = [[dimod.Binary(f'x_tower_#{j}_{t}') for t in range(T)] for j in range(J_tower)]
x_tower_coefficient = np.zeros((J_tower, T))

t = [dimod.Binary(f't_{i}') for i in range(t_bits)]
t_coefficient = np.power(2, np.arange(-t_decimal,t_bits - t_decimal), dtype=float)



#objective function 
cqm.set_objective(sum(t[i]*t_coefficient[i] for i in range(t_bits)) + \
                  sum(sum(u[i][t] * u_coefficient[i,t] for t in range(T)) for i in range(len(u_name_iterable))) + \
                  sum(sum(x_chiller[j][t] * x_chiller_coefficient[j,t] for t in range(T)) for j in range(J_chiller)) + \
                  sum(sum(x_tower[j][t] * x_tower_coefficient[j,t] for t in range(T)) for j in range(J_tower)))


# record binary variable position
binary_desiction_dict = {}
binary_desiction_val_dict = {}
problem_A_dict = {}
index_counter = 0

for position, binary_name in enumerate(cqm.variables):
    binary_desiction_dict.update({binary_name: position})
    binary_desiction_val_dict.update({binary_name: 0})
    
    if not binary_name.split("_")[0] == "t":
        problem_A_dict.update({binary_name: index_counter})
        index_counter += 1
        
#setup masterproblem x vector:
master_x_sol = np.zeros(len(problem_A_dict))
    
# record continuous variable position
continuous_decision_list_T = ["p_dis","p_chr","v_vent","e_in"]
cont_decision_index_counter = 0
problem_G_dict = {}

# arrage the position accordingly to decision var with len t
for var_name in continuous_decision_list_T:
    for t in range(T):
        problem_G_dict.update({f"{var_name}_{t}":  cont_decision_index_counter})
        cont_decision_index_counter += 1
##"T_zone"   "T_sup_zone"
for zone in range(I_zone):
    ##"T_zone"
    for t in range(T + 1):
        problem_G_dict.update({f"T_zone_{zone}_{t}":  cont_decision_index_counter})
        cont_decision_index_counter += 1
        
    ##  "T_sup_zone"
    for t in range(T):
        problem_G_dict.update({f"T_sup_zone_{zone}_{t}":  cont_decision_index_counter})
        cont_decision_index_counter += 1
        
## "E_battery_state"
for t in range(T + 1):
        problem_G_dict.update({f"E_battery_state_{t}":  cont_decision_index_counter})
        cont_decision_index_counter += 1

    

    
problem_constriant_dict = {}    
problem_constraint = 18
problem_constraint_counter = 0
for line in range(problem_constraint):
    # constraint 10 ~ 12  has i * t constaint
    if line in [9,10,11,16,17]:
        for i in range(I_zone):
            for t in range(T):
        
                problem_constriant_dict.update({f"constraint_{line}_zone_{i}_{t}":  problem_constraint_counter })
        
                problem_constraint_counter += 1
                
    # contraint 4 initalization
    elif line == 3:
        problem_constriant_dict.update({f"constraint_{line}":  problem_constraint_counter})
        
        problem_constraint_counter += 1
    
    # contraint 16 initalization
    elif line == 15:
        for i in range(I_zone):
            problem_constriant_dict.update({f"constraint_{line}_zone_{i}":  problem_constraint_counter })
        
            problem_constraint_counter += 1
        
    #other constarints
    else:
        for t in range(T):
        
            problem_constriant_dict.update({f"constraint_{line}_{t}":  problem_constraint_counter })
        
            problem_constraint_counter += 1
    
    
    
    
#print(binary_desiction_dict)
print(problem_G_dict)


########################################################################
#   
#    START construct d
#
########################################################################

d = np.zeros(len(problem_G_dict))

for t in range(T):
    d[problem_G_dict[f'e_in_{t}']] = 1

########################################################################
#   
#    START construct A, G, b
#
########################################################################

# contrust matrix A
A = np.zeros((len(problem_constriant_dict ),len(problem_A_dict)))
print(A.shape)

for t in range(T):
    # (1st constraint)
    
    for j in range(J_chiller):
        A[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_A_dict[f"x_chiller_#{j}_{t}"]] = -beta_chiller_0[j] - beta_chiller_1[j] * M_chiller_rated[j] 
    for j in range(J_tower):
        A[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_A_dict[f"x_tower_#{j}_{t}"]] = -beta_tower_0[j] - beta_tower_1[j] * M_tower_rated[j] 

    # (7th constraints)
    
    A[problem_constriant_dict[f"constraint_{6}_{t}"] , problem_A_dict[f"u_char_{t}"]] = char_b_upperbound 
    
    # (8th constraints)
    
    A[problem_constriant_dict[f"constraint_{7}_{t}"] , problem_A_dict[f"u_dischar_{t}"]] = dis_b_upperbound 
    
    # (9th constraints)
    
    index_of_dis_and_chr = [problem_A_dict[f"u_char_{t}"] , problem_A_dict[f"u_dischar_{t}"]]
    A[problem_constriant_dict[f"constraint_{8}_{t}"],index_of_dis_and_chr] = -1
    
    # (14th constraints)
    for j in range(J_chiller):
        A[problem_constriant_dict[f"constraint_{13}_{t}"],problem_A_dict[f"x_chiller_#{j}_{t}"]] = M_chiller_rated[j] 
        
    # (15th constraints)
    for j in range(J_tower):
        A[problem_constriant_dict[f"constraint_{14}_{t}"],problem_A_dict[f"x_tower_#{j}_{t}"]] = M_tower_rated[j] 
        
# contrust matrix G
G = np.zeros((len(problem_constriant_dict ),len(problem_G_dict)))

# initial setup
# (4th constraint)
G[problem_constriant_dict[f"constraint_{3}"] , problem_G_dict["E_battery_state_0"]] = Battery_capacity_init

# 16th constraint)
for zone in range(I_zone):
    G[problem_constriant_dict[f"constraint_{15}_zone_{zone}"] , problem_G_dict[f"T_zone_{zone}_0"]] = 1 
    
for t in range(T):
    # (1st constraint)
    G[problem_constriant_dict[f"constraint_{0}_{t}"],problem_G_dict[f"p_dis_{t}"]] = 1/eta_dischar_b
    G[problem_constriant_dict[f"constraint_{0}_{t}"],problem_G_dict[f"p_chr_{t}"]] = -eta_char_b
    for zone in range(I_zone):
        G[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - beta_pump_1 * Chi[zone] * (v_sup[t] - v_out[t]) * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
        G[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = beta_pump_1 * Chi[zone] * v_sup[t] * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
        G[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_G_dict[f"v_vent_{t}"]] = - beta_vent_1
        
    # (3rd constraint)
    G[problem_constriant_dict[f"constraint_{2}_{t}"] ,problem_G_dict[f"p_dis_{t}"]] = 1/eta_dischar_b
    G[problem_constriant_dict[f"constraint_{2}_{t}"] ,problem_G_dict[f"p_chr_{t}"]] = -eta_char_b
    G[problem_constriant_dict[f"constraint_{2}_{t}"],problem_G_dict[f"E_battery_state_{t}"]] = -1
    G[problem_constriant_dict[f"constraint_{2}_{t}"],problem_G_dict[f"E_battery_state_{t + 1}"]] = 1
    
    # (10th constraint)
    for zone in range(I_zone):
        G[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - 1 + np.sum(adjacent_zone_map[zone , :]) /(C_air_normal * R_air_ij) + m_air_in[zone] * c_s_air[zone] / C_air_normal
        
        G[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = 1
        
        G[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = - m_air_in[zone] * c_s_air[zone] / C_air_normal
        
        adjacent_zone = np.where(adjacent_zone_map[zone ,:])[0]
        for adj_zone in adjacent_zone:
            G[problem_constriant_dict[f"constraint_{9}_zone_{adj_zone}_{t}"] , problem_G_dict[f"T_zone_{adj_zone}_{t}"]] = - 1 / (C_air_normal * R_air_ij)
            
    # (2nd constraint)
    G[problem_constriant_dict[f"constraint_{1}_{t}"] , problem_G_dict[f"e_in_{t}"]] = 1
    
    # (5th constraint)
    G[problem_constriant_dict[f"constraint_{4}_{t}"] , problem_G_dict[f"E_battery_state_{t + 1}"]] = -1
    
    # (6th constraint)
    G[problem_constriant_dict[f"constraint_{5}_{t}"] , problem_G_dict[f"E_battery_state_{t + 1}"]] = 1
    
    # (7th constraint)
    G[problem_constriant_dict[f"constraint_{6}_{t}"] , problem_G_dict[f"p_chr_{t}"]] = 1
    
    # (8th constraint)
    G[problem_constriant_dict[f"constraint_{7}_{t}"] , problem_G_dict[f"p_dis_{t}"]] = 1
    
    
    for zone in range(I_zone):
        
        # (11th constraint)
        G[problem_constriant_dict[f"constraint_{10}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = - 1
    
        # (12th constraint)
        G[problem_constriant_dict[f"constraint_{11}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = 1
        
        # (11a "17" th constraint)
        G[problem_constriant_dict[f"constraint_{16}_zone_{zone}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = -1
        
        # (12a "18" th constraint)
        G[problem_constriant_dict[f"constraint_{17}_zone_{zone}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = 1
        
    # (13th constraint)
    G[problem_constriant_dict[f"constraint_{12}_{t}"] , problem_G_dict[f"v_vent_{t}"]] = 1
    
    
    
    for zone in range(I_zone):
        
        # (14th constraint)
        G[problem_constriant_dict[f"constraint_{13}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - Chi[zone] * (v_sup[t] - v_out[t]) * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
        G[problem_constriant_dict[f"constraint_{13}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = Chi[zone] * v_sup[t] * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
        
        # (15th constraint)
        G[problem_constriant_dict[f"constraint_{14}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - Chi[zone] * (v_sup[t] - v_out[t]) * C_air_normal / ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water)
        G[problem_constriant_dict[f"constraint_{14}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = Chi[zone] * v_sup[t] * C_air_normal / ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water)

    
# contrust vector b
b = np.zeros((len(problem_constriant_dict ),1))

# (4rd constraint)
b[problem_constriant_dict[f"constraint_{3}"],0] = Battery_capacity_init

# (16th constraint)
for zone in range(I_zone):
    b[problem_constriant_dict[f"constraint_{15}_zone_{zone}"] , 0] = T_zone_initial[zone//I_zone_length , zone%I_zone_length]

for t in range(T):
    # (1st constraint)
    b[problem_constriant_dict[f"constraint_{0}_{t}"],0] = e_sup[t] + beta_vent_0  - beta_vent_1 * v_vent_lowerbound[t] + \
    beta_pump_0 + beta_pump_1 * T_out[t] * v_sup[t]  * C_air_normal/ ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water) + e_server[t] + e_solar[t]
    
    # (3rd constraint)
    b[problem_constriant_dict[f"constraint_{2}_{t}"],0] = 0
    
    # (10th constraint)
    for zone in range(I_zone):
        b[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , 0] = theta_heat[zone,t]/c_s_air[zone]
            
    # (2nd constraint)
    b[problem_constriant_dict[f"constraint_{1}_{t}"] , 0] = 0
    
    # (5th constraint)
    b[problem_constriant_dict[f"constraint_{4}_{t}"] , 0] = -Battery_capacity_upbound 
    
    # (6th constraint)
    b[problem_constriant_dict[f"constraint_{5}_{t}"] , 0] = Battery_capacity_lowbound
    
    # (7th constraint)
    b[problem_constriant_dict[f"constraint_{6}_{t}"] , 0] = 0
    
    # (8th constraint)
    b[problem_constriant_dict[f"constraint_{7}_{t}"] , 0] = 0
      
    # (9th constraint)
    b[problem_constriant_dict[f"constraint_{8}_{t}"] , 0] = -1
    
    
    for zone in range(I_zone):
        
        # (11th constraint)
        b[problem_constriant_dict[f"constraint_{10}_zone_{zone}_{t}"] , 0] = -T_datacenter_upperbound[zone , t]
    
        # (12th constraint)
        b[problem_constriant_dict[f"constraint_{11}_zone_{zone}_{t}"] , 0] = T_datacenter_lowerbound[zone , t]
        
        # (11a "17" th constraint)
        b[problem_constriant_dict[f"constraint_{16}_zone_{zone}_{t}"] , 0] = - T_AC_upperbound[zone , t]
        
        # (12a "18" th constraint)
        b[problem_constriant_dict[f"constraint_{17}_zone_{zone}_{t}"] , 0] = T_AC_lowerbound[zone , t]
        
    # (13th constraint)
    b[problem_constriant_dict[f"constraint_{12}_{t}"] , 0] = v_vent_lowerbound[t]
    
    
    for zone in range(I_zone):
        
        # (14th constraint)
        b[problem_constriant_dict[f"constraint_{13}_{t}"] , 0] = - T_out[t] * v_sup[t]* C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
        
        # (15th constraint)
        b[problem_constriant_dict[f"constraint_{14}_{t}"] , 0] = - T_out[t] * v_sup[t] * C_air_normal / ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water)

{'p_dis_0': 0, 'p_dis_1': 1, 'p_dis_2': 2, 'p_dis_3': 3, 'p_chr_0': 4, 'p_chr_1': 5, 'p_chr_2': 6, 'p_chr_3': 7, 'v_vent_0': 8, 'v_vent_1': 9, 'v_vent_2': 10, 'v_vent_3': 11, 'e_in_0': 12, 'e_in_1': 13, 'e_in_2': 14, 'e_in_3': 15, 'T_zone_0_0': 16, 'T_zone_0_1': 17, 'T_zone_0_2': 18, 'T_zone_0_3': 19, 'T_zone_0_4': 20, 'T_sup_zone_0_0': 21, 'T_sup_zone_0_1': 22, 'T_sup_zone_0_2': 23, 'T_sup_zone_0_3': 24, 'T_zone_1_0': 25, 'T_zone_1_1': 26, 'T_zone_1_2': 27, 'T_zone_1_3': 28, 'T_zone_1_4': 29, 'T_sup_zone_1_0': 30, 'T_sup_zone_1_1': 31, 'T_sup_zone_1_2': 32, 'T_sup_zone_1_3': 33, 'T_zone_2_0': 34, 'T_zone_2_1': 35, 'T_zone_2_2': 36, 'T_zone_2_3': 37, 'T_zone_2_4': 38, 'T_sup_zone_2_0': 39, 'T_sup_zone_2_1': 40, 'T_sup_zone_2_2': 41, 'T_sup_zone_2_3': 42, 'T_zone_3_0': 43, 'T_zone_3_1': 44, 'T_zone_3_2': 45, 'T_zone_3_3': 46, 'T_zone_3_4': 47, 'T_sup_zone_3_0': 48, 'T_sup_zone_3_1': 49, 'T_sup_zone_3_2': 50, 'T_sup_zone_3_3': 51, 'T_zone_4_0': 52, 'T_zone_4_1': 53, 'T_zone_4_2': 54, 'T_

In [116]:
#sampler = LeapHybridCQMSampler()    

#sampleset = sampler.sample_cqm(cqm, label="SDK Examples - Solar Panel Strategy")
    
#best = sampleset.filter(lambda row: row.is_feasible).first

#print("{} feasible solutions of {}.".format(sampleset.record.is_feasible.sum(), len(sampleset))) 

#locals()[""]

In [117]:
counter = 0
t_upper = np.inf
t_lower = -np.inf
threshold = 0.5

for key, val in best.sample.items():
    #print(key, val)
    if key.split("_")[0]=="t":
        t_lower += val * t_coefficient[int(key.split("_")[-1])]
    else:
        master_x_sol[problem_A_dict[key]] = val
    
    binary_desiction_val_dict.update({key: val})
        

In [118]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

master_x_sol = np.zeros(60)
print(master_x_sol)

infunbdinfo=1
rhs_vector = b - np.dot(A , master_x_sol.flatten())
# create one model instance, with a name
DCMQG = gp.Model("Data_center_quantum_model")

#DCMQG.setParam("DualReductions", 0)
DCMQG.setParam("InfUnbdInfo", 1)

u_sub = DCMQG.addVars(len(rhs_vector), vtype=GRB.CONTINUOUS, name='u')

# No. 1 3 4 10 16 Constriants has no upper/lower bound


for t in range(T):                
    # (2nd constraint)
    u_sub[problem_constriant_dict[f"constraint_{1}_{t}"]].lb = 0
    
    # (5th constraint)
    u_sub[problem_constriant_dict[f"constraint_{4}_{t}"]].lb = 0
    
    # (6th constraint)
    u_sub[problem_constriant_dict[f"constraint_{5}_{t}"]].lb = 0
    
    # (7th constraint)
    u_sub[problem_constriant_dict[f"constraint_{6}_{t}"]].lb = 0
    
    # (8th constraint)
    u_sub[problem_constriant_dict[f"constraint_{7}_{t}"]].lb = 0
      
    # (9th constraint)
    u_sub[problem_constriant_dict[f"constraint_{8}_{t}"]].lb = 0
    
    
    for zone in range(I_zone):
        
        # (11th constraint)
        u_sub[problem_constriant_dict[f"constraint_{10}_zone_{zone}_{t}"]].lb = 0
    
        # (12th constraint)
        u_sub[problem_constriant_dict[f"constraint_{11}_zone_{zone}_{t}"]].lb = 0
        
        # (11a "17" th constraint)
        u_sub[problem_constriant_dict[f"constraint_{16}_zone_{zone}_{t}"]].lb = 0
        
        # (12a "18" th constraint)
        u_sub[problem_constriant_dict[f"constraint_{17}_zone_{zone}_{t}"]].lb = 0
        
    # (13th constraint)
    u_sub[problem_constriant_dict[f"constraint_{12}_{t}"]].lb = 0
    
    
    for zone in range(I_zone):
        
        # (14th constraint)
        u_sub[problem_constriant_dict[f"constraint_{13}_{t}"]].lb = 0
        
        # (15th constraint)
        u_sub[problem_constriant_dict[f"constraint_{14}_{t}"]].lb = 0
        


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Set parameter InfUnbdInfo to value 1


In [119]:
# objective function
rhs = rhs_vector.flatten()

DCMQG.setObjective(quicksum( rhs[i] * u_sub[i] for i in range(len(rhs_vector))), GRB.MAXIMIZE)
G_transpose = G.T
print(G_transpose.shape,len(u_sub),len(d))
u_height, u_width = G_transpose.shape

# np.dot(G.T, u) < h
for i in range(u_height):
    DCMQG.addConstr(quicksum(G_transpose[i][j] * u_sub[j] for j in range(u_width)) >= d[i], name=f'Subproblem_constraints_{i}')
        
DCMQG_sol = DCMQG.optimize()
#assert DCMG_sol
print (DCMQG.display())       
#DCMQG.display()

(102, 234) 234 102
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads
Optimize a model with 102 rows, 234 columns and 494 nonzeros
Model fingerprint: 0xe5115b0b
Coefficient statistics:
  Matrix range     [1e-03, 8e+01]
  Objective range  [1e+03, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0404702e+36   2.588845e+31   5.040470e+06      0s

Solved in 55 iterations and 0.00 seconds (0.00 work units)
Unbounded model
Maximize
<gurobi.LinExpr: 1037.4940219990435 u[0] + 1037.4940219990435 u[1]
+ 1037.4940219990435 u[2] + 1037.4940219990435 u[3] + 1037.4940219990435 u[4]
+ 1037.4940219990435 u[5] + 1037.4940219990435 u[6] + 1037.4940219990435 u[7]
+ 1037.4940219990435 u[8] + 1037.4940219990435 u[9] + 1037.4940219990435 u[10]
+ 1037.4940219990435 u[11] + 1037.4940219990435 u[12] + 1037

  Subproblem_constraints_13: <gurobi.LinExpr: u[5]> >= 1
  Subproblem_constraints_14: <gurobi.LinExpr: u[6]> >= 1
  Subproblem_constraints_15: <gurobi.LinExpr: u[7]> >= 1
Subproblem_constraints_16: <gurobi.LinExpr: -0.009538232637228333 u[0] +
-0.0033160896670645975 u[33] + -0.0009538232637228332 u[145] + -0.0009538232637228332
 u[149] + u[153]> >= 0
Subproblem_constraints_17: <gurobi.LinExpr: -0.009538232637228333 u[1] + -75.0 u[33] +
-0.0033160896670645975 u[34] + -1.0 u[69] + u[105] + -0.0009538232637228332 u[146] +
 -0.0009538232637228332 u[150]> >= 0
Subproblem_constraints_18: <gurobi.LinExpr: -0.009538232637228333 u[2] + -75.0 u[34] +
-0.0033160896670645975 u[35] + -1.0 u[70] + u[106] + -0.0009538232637228332 u[147] +
 -0.0009538232637228332 u[151]> >= 0
Subproblem_constraints_19: <gurobi.LinExpr: -0.009538232637228333 u[3] + -75.0 u[35] +
-0.0033160896670645975 u[36] + -1.0 u[71] + u[107] + -0.0009538232637228332 u[148] +
 -0.0009538232637228332 u[152]> >= 0
  Subproblem_constra

 -0.0009538232637228332 u[150]> >= 0
Subproblem_constraints_63: <gurobi.LinExpr: -0.009538232637228333 u[2] + -75.0 u[54] +
-0.0033160896670645975 u[55] + -1.0 u[90] + u[126] + -0.0009538232637228332 u[147] +
 -0.0009538232637228332 u[151]> >= 0
Subproblem_constraints_64: <gurobi.LinExpr: -0.009538232637228333 u[3] + -75.0 u[55] +
-0.0033160896670645975 u[56] + -1.0 u[91] + u[127] + -0.0009538232637228332 u[148] +
 -0.0009538232637228332 u[152]> >= 0
  Subproblem_constraints_65: <gurobi.LinExpr: -75.0 u[56] + -1.0 u[92] + u[128]> >= 0
Subproblem_constraints_66: <gurobi.LinExpr: 0.019076465274456667 u[0] +
 0.0019076465274456664 u[145] + 0.0019076465274456664 u[149] + -1.0 u[182] + u[218]> >= 0
Subproblem_constraints_67: <gurobi.LinExpr: 0.019076465274456667 u[1] +
 0.0019076465274456664 u[146] + 0.0019076465274456664 u[150] + -1.0 u[183] + u[219]> >= 0
Subproblem_constraints_68: <gurobi.LinExpr: 0.019076465274456667 u[2] +
 0.0019076465274456664 u[147] + 0.0019076465274456664 u[151] + 

In [120]:
print(DCMQG.getAttr(GRB.Attr.Status))


5


In [121]:
u_sol = DCMQG.getVars()
u_sol_ray = DCMQG.getAttr(GRB.Attr.UnbdRay, DCMQG.getVars())

In [122]:
#print('Obj: %g' % DCMQG.ObjVal)

for v in u_sol:
    print('%s %g' % (v.VarName, v.X))
        
#for v in u_sol_ray:
#    print('%s %g' % (v.VarName, v.X))
#len(u_sol_ray)

#print([item.X for item in u_sol], u_sol_ray)
result_ray = np.array([item.X for item in u_sol]) 

u[0] 0
u[1] 0
u[2] 0
u[3] 0
u[4] 1
u[5] 1
u[6] 1
u[7] 1
u[8] 0
u[9] 0
u[10] 0
u[11] 0
u[12] 0
u[13] 0
u[14] 0
u[15] 0
u[16] 0
u[17] 0
u[18] 0
u[19] 0
u[20] 0
u[21] 0
u[22] 0
u[23] 0
u[24] 0
u[25] 0
u[26] 0
u[27] 0
u[28] 0
u[29] 0
u[30] 0
u[31] 0
u[32] 0
u[33] 0
u[34] 0
u[35] 0
u[36] 0
u[37] 0
u[38] 0
u[39] 0
u[40] 0
u[41] 0
u[42] 0
u[43] 0
u[44] 0
u[45] 0
u[46] 0
u[47] 0
u[48] 0
u[49] 0
u[50] 0
u[51] 0
u[52] 0
u[53] 0
u[54] 0
u[55] 0
u[56] 0
u[57] 0
u[58] 0
u[59] 0
u[60] 0
u[61] 0
u[62] 0
u[63] 0
u[64] 0
u[65] 0
u[66] 0
u[67] 0
u[68] 0
u[69] 0
u[70] 0
u[71] 0
u[72] 0
u[73] 0
u[74] 0
u[75] 0
u[76] 0
u[77] 0
u[78] 0
u[79] 0
u[80] 0
u[81] 0
u[82] 0
u[83] 0
u[84] 0
u[85] 0
u[86] 0
u[87] 0
u[88] 0
u[89] 0
u[90] 0
u[91] 0
u[92] 0
u[93] 0
u[94] 0
u[95] 0
u[96] 0
u[97] 0
u[98] 0
u[99] 0
u[100] 0
u[101] 0
u[102] 0
u[103] 0
u[104] 0
u[105] 0
u[106] 0
u[107] 0
u[108] 0
u[109] 0
u[110] 0
u[111] 0
u[112] 0
u[113] 0
u[114] 0
u[115] 0
u[116] 0
u[117] 0
u[118] 0
u[119] 0
u[120] 0
u[121] 0
u[122] 0
u[1

In [123]:
#print(b,result_ray)
#print(np.dot(result_ray, b) )

print(np.dot(result_ray, A) )

Master_problem_cons = {}


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [124]:
A

array([[     0.,      0.,      0., ...,      0.,      0.,      0.],
       [     0.,      0.,      0., ..., -40060.,      0.,      0.],
       [     0.,      0.,      0., ...,      0., -40060.,      0.],
       ...,
       [     0.,      0.,      0., ...,      0.,      0.,      0.],
       [     0.,      0.,      0., ...,      0.,      0.,      0.],
       [     0.,      0.,      0., ...,      0.,      0.,      0.]])

In [125]:
# initialization
counter = 0
t_upper = np.inf
t_lower = -np.inf
threshold = 0.5


while((counter < 1) and (abs(t_upper - t_lower) > threshold)):
    print("counter:" + str(counter))
    
    sampler = LeapHybridCQMSampler()    

    sampleset = sampler.sample_cqm(cqm, label="SDK Examples - Solar Panel Strategy")
    
    best = sampleset.filter(lambda row: row.is_feasible).first
    
    
    
    
    
    counter+= 1

counter:0


In [126]:
t_lower = 0
for key, val in best.sample.items():
    #print(key, val)
    if key.split("_")[0]=="t":
        t_lower += val * t_coefficient[int(key.split("_")[-1])]
    
binary_desiction_val_dict.update({key: val})
        
print(binary_desiction_val_dict,t_lower)

{'t_0': 0.0, 't_1': 0.0, 't_2': 0.0, 't_3': 0.0, 't_4': 0.0, 't_5': 0.0, 't_6': 0.0, 't_7': 0.0, 't_8': 0.0, 't_9': 0.0, 't_10': 0.0, 't_11': 0.0, 't_12': 0.0, 't_13': 0.0, 't_14': 0.0, 't_15': 0.0, 't_16': 0.0, 't_17': 0.0, 't_18': 0.0, 't_19': 0.0, 't_20': 0.0, 't_21': 0.0, 't_22': 0.0, 't_23': 0.0, 't_24': 0.0, 't_25': 0.0, 't_26': 0.0, 't_27': 0.0, 't_28': 0.0, 't_29': 0.0, 't_30': 0.0, 't_31': 0.0, 't_32': 0.0, 't_33': 0.0, 't_34': 0.0, 't_35': 0.0, 't_36': 0.0, 't_37': 0.0, 't_38': 0.0, 't_39': 0.0, 'u_dischar_0': 0.0, 'u_dischar_1': 0.0, 'u_dischar_2': 0.0, 'u_dischar_3': 1.0, 'u_char_0': 0.0, 'u_char_1': 0.0, 'u_char_2': 1.0, 'u_char_3': 1.0, 'x_chiller_#0_0': 0.0, 'x_chiller_#0_1': 1.0, 'x_chiller_#0_2': 0.0, 'x_chiller_#0_3': 1.0, 'x_chiller_#1_0': 0.0, 'x_chiller_#1_1': 0.0, 'x_chiller_#1_2': 0.0, 'x_chiller_#1_3': 1.0, 'x_chiller_#2_0': 1.0, 'x_chiller_#2_1': 1.0, 'x_chiller_#2_2': 1.0, 'x_chiller_#2_3': 0.0, 'x_chiller_#3_0': 1.0, 'x_chiller_#3_1': 0.0, 'x_chiller_#3_2': 1

In [83]:
t_lower = []

<function dict.items>